## Import related functions

In [7]:
import cobra
import re 
#from script.ECMpy_function import *
import sys
sys.path.append(r'./script/')
from ECMpy_function import *
import numpy as np

## Input and output files

In [10]:
# reaction kcat_mw
sbml_path = "./data/iZM547.xml"
taxonom_id=264203
method='AutoPACMEN' #AutoPACMEN #AutoPACMEN_mean #DLkcat # TurNup # UniKP
reaction_kcat_MW_file = "./analysis/get_kcat_mw_by_%s/reaction_kcat_MW.csv"%method
#paxdb丰度数据
# gene_abundance_colname='abundance(ppm)'
# gene_abundance_file='./data/PXD030417_protein_abundance.abu' # calculated from https://pax-db.org/download
# #The enzyme mass fraction,calculated as 0.45387051337830087  proteome calculation result
f = 0.45387051337830087  # 蛋白组计算
#Initial parameters
ptot = 0.605 # The total protein fraction in cell.
sigma = 0.5 # The approximated saturation of enzyme.e.g.,0.5/1.
lowerbound = 0   # Lowerbound  of enzyme concentration constraint. 
upperbound = round(ptot * f * sigma, 3)#total enzyme
concentration = 40 # glucose uptake rate of q = 41.8 mmol/gdw/h   10.1016/j.ymben.2019.05.006
ecModel_output_file="./model/iZM547_"+ method +".json"

## Get ecModel and simulation

In [11]:
#Get ecModel
trans_model2enz_json_model_split_isoenzyme(sbml_path, reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, ecModel_output_file)

#ecModel Simulation
obj='ZM_BIOMASS'# CG_biomass_cgl_ATCC13032 EX_lys_L_e
fluxes_outfile = "./analysis/"+ method+("ECMpy_solution_%s_pfba.csv"%obj)
use_substrate='EX_glc_D_e'
enz_model=get_enzyme_constraint_model(ecModel_output_file)
enz_model.objective=obj

# change original substrate in model
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, concentration)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
# enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_output_file)
print(enz_model_pfba_solution.fluxes[obj])

# dlkcat 转录组0.03326546325676637 蛋白组 0.03137498657327576

0.5216633885328134
